# **FuDgetron** **Assignment**

**Q.1 Write python code that would do the following:**

In [4]:
#importing necessary libraries

import pandas as pd
from geopy.distance import geodesic
from geopy.geocoders import ArcGIS

I.	Create a data frame from the input (File: Delhi-Electricity-SubStation.csv).

In [5]:
#createing a data frame

df = pd.read_csv('/content/Delhi-Electricity-SubStations.csv')

In [6]:
#displaying the first few rows of a DataFrame

df.head()


,_id,Substations,Telephone Numbers,Address,Voltage Class,Coordinates
0,1,Bawana,27791190/1193/1210,"400kV Sub-Station Bawana, Sector-5, DSIIDC Baw...",400 kV,"28.79568, 77.0723"
1,2,Bamnauli,25314199/25314204,"Village-Bamnauli, P.O. Dhul Sirus, Near Chhawa...",400 kV,"28.5447, 77.03269"
2,3,Harsh Vihar,0120-6500138,"Harsh Vihar, Loni Road (Near Bhopura Chowk), D...",400 kV,"28.71185, 77.29044"
3,4,Tikri Kalan(Mundka),65108444/7290010893,"Neewala Village Road, Near Vaishno Devi Mandir...",400 kV,"28.67671, 76.98639"
4,5,BTPS,26948637,"Badarpur Thermal Power Station Complex, Badarp...",220 kV,"28.50765, 77.30015"


In [7]:
#getting summary of the DataFrame

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   _id                43 non-null     int64 
 1   Substations        43 non-null     object
 2   Telephone Numbers  40 non-null     object
 3   Address            43 non-null     object
 4   Voltage Class      43 non-null     object
 5   Coordinates        41 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.1+ KB


In [5]:
#dropping the rows with missing values (NaN/null values) from a DataFrame

df.dropna(subset=['Telephone Numbers','Coordinates'], inplace=True)

II.	In Telephone Numbers there are multiple numbers.  Create separate columns for these Telephone1, Telephone2 etc.

In [8]:
# Spliting Telephone Numbers into separate columns

split_numbers = df['Telephone Numbers'].str.split('/', expand=True)

In [9]:
df['Telephone1'] = split_numbers[0]
df['Telephone2'] = split_numbers[1]
df['Telephone3'] = split_numbers[2]

In [10]:
df.head()

,_id,Substations,Telephone Numbers,Address,Voltage Class,Coordinates,Telephone1,Telephone2,Telephone3
0,1,Bawana,27791190/1193/1210,"400kV Sub-Station Bawana, Sector-5, DSIIDC Baw...",400 kV,"28.79568, 77.0723",27791190,1193,1210
1,2,Bamnauli,25314199/25314204,"Village-Bamnauli, P.O. Dhul Sirus, Near Chhawa...",400 kV,"28.5447, 77.03269",25314199,25314204,None
2,3,Harsh Vihar,0120-6500138,"Harsh Vihar, Loni Road (Near Bhopura Chowk), D...",400 kV,"28.71185, 77.29044",0120-6500138,None,None
3,4,Tikri Kalan(Mundka),65108444/7290010893,"Neewala Village Road, Near Vaishno Devi Mandir...",400 kV,"28.67671, 76.98639",65108444,7290010893,None
4,5,BTPS,26948637,"Badarpur Thermal Power Station Complex, Badarp...",220 kV,"28.50765, 77.30015",26948637,None,None


III.	Determine the accuracy of the Coordinates with Address.  Add another column GeoAccuracy.  Find the coordinates using the address.  If the coordinates match exactly with address, then accuracy is 100%, otherwise for every 10 meters off consider a drop in accuracy of 1 %.  For example, if the coordinate of the address is 30 meters away from the given coordinates, the accuracy would be 70%.

In [12]:

#converting addresses into geographic coordinates


def geocode_address(address):
    nom = ArcGIS()
    coordinate = nom.geocode(address)
    if coordinate:
        rounded_tuple = tuple(round(value, 5) for value in coordinate[1])
        return rounded_tuple
    else:
        return None


In [13]:
# calculating accuracy
def calculate_accuracy(row):
    if pd.notnull(row['Coordinates']):
        given_coords = tuple(map(float, row['Coordinates'].split(',')))
        address_coords = geocode_address(row['Address'])

        if address_coords:
            distance = geodesic(given_coords, address_coords).meters

            if distance == 0:
                return 100.0
            else:
                accuracy = max(0, 100 - (distance / 10))
                return round(accuracy, 2)
        else:
            return None
    else:
        return None


In [14]:
# Applying the calculate_accuracy function to each row

df['GeoAccuracy'] = df.apply(calculate_accuracy, axis=1)

In [15]:
df


,_id,Substations,Telephone Numbers,Address,Voltage Class,Coordinates,Telephone1,Telephone2,Telephone3,GeoAccuracy
0,1,Bawana,27791190/1193/1210,"400kV Sub-Station Bawana, Sector-5, DSIIDC Baw...",400 kV,"28.79568, 77.0723",27791190,1193,1210,0.00
1,2,Bamnauli,25314199/25314204,"Village-Bamnauli, P.O. Dhul Sirus, Near Chhawa...",400 kV,"28.5447, 77.03269",25314199,25314204,None,23.70
2,3,Harsh Vihar,0120-6500138,"Harsh Vihar, Loni Road (Near Bhopura Chowk), D...",400 kV,"28.71185, 77.29044",0120-6500138,None,None,0.00
3,4,Tikri Kalan(Mundka),65108444/7290010893,"Neewala Village Road, Near Vaishno Devi Mandir...",400 kV,"28.67671, 76.98639",65108444,7290010893,None,0.00
4,5,BTPS,26948637,"Badarpur Thermal Power Station Complex, Badarp...",220 kV,"28.50765, 77.30015",26948637,None,None,89.41
5,6,DSIDC Bawana,65005603/7290013476,"Near H Block, J.J. colony, Bawana Hanuman Mand...",220 kV,"28.80361, 77.05284",65005603,7290013476,None,87.53
6,7,DIAL,25655090,"Near IGI Airport, Delhi",220 kV,"28.55616, 77.09995",25655090,None,None,0.00
7,8,Electric Lane,23329790,"HCM Lane, Behind BSNL Building, Janpath, New D...",220 kV,"28.62174, 77.21969",23329790,None,None,9.78
8,9,Geeta Colony,22044528,"Near Sai Memorial Public School, Geeta Colony,...",220 kV,"28.65226, 77.26774",22044528,None,None,25.70
9,10,Gazipur,22770009,"Village Gazipur, Near DDA Janta Flat, Delhi-11...",220 kV,"28.62961, 77.31898",22770009,None,None,56.42


**Q.2 The file MH-Veh-Reg.csv contains data of vehicles registered in Maharashtra for the period 2000-2018.   Write python code to do the following**

In [16]:
#creating the dataframe


df_vehicles = pd.read_csv('/content/MH-Vehicle-Reg.csv',sep=';')

In [19]:
df_vehicles.head()

,_id,Sr No.,Year,Region,Sub Region,Motor Cycles,Scooters,Moped,Cars,Jeeps,...,Private Service Vehicles,Ambulances,Articulated/Multi.,Trucks & Lorries,Tanker,Delivery Van (4 wheelers),Delivery Van (3 wheelers),Tractors,Trailors,Others
0,1,1,2000-2001,Greater Mumbai,Mumbai(C),84289,62444,7289,164758,12727,...,829,902,0,9022,1214,9329,4079,867,866,3761
1,2,2,2000-2001,Greater Mumbai,Mumbai(W),80320,96297,15230,110397,5465,...,605,298,0,5637,117,5019,9517,370,127,445
2,3,3,2000-2001,Greater Mumbai,Mumbai(E),39930,44932,9786,42945,4439,...,279,158,0,3811,697,4180,4004,124,170,577
3,4,4,2000-2001,Greater Mumbai,Borivali,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,5,2000-2001,Thane Region,Thane,130448,104028,9304,96933,23755,...,1030,206,0,42289,8159,21309,8228,1107,4352,740


In [18]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   _id                           900 non-null    int64 
 1   Sr No.                        900 non-null    int64 
 2   Year                          900 non-null    object
 3   Region                        900 non-null    object
 4   Sub Region                    900 non-null    object
 5   Motor Cycles                  900 non-null    int64 
 6   Scooters                      900 non-null    int64 
 7   Moped                         900 non-null    int64 
 8   Cars                          900 non-null    int64 
 9   Jeeps                         900 non-null    int64 
 10  Stn. Wagons                   900 non-null    int64 
 11  Taxis meter fited             900 non-null    int64 
 12  Luxury /Turist Cabs/          900 non-null    int64 
 13  Auto-rikshaws       

i.	Merge the data of each year (2000-2001, 2001-2002,….) to a single period (2000-2018) by the region and sub region.

In [23]:
start_year = 2000
end_year = 2018

# Creating  new 'Period' column with the entire range (2000-2018)
df_vehicles['Period'] = f"{start_year}-{end_year}"

In [24]:
# Groupping the data by 'Region', 'Sub Region', and 'Period', summing the values
merged_df = df_vehicles.groupby(['Region', 'Sub Region', 'Period']).sum().reset_index()

<ipython-input-24-339487c163e0>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_df = df_vehicles.groupby(['Region', 'Sub Region', 'Period']).sum().reset_index()


In [25]:
merged_df

,Region,Sub Region,Period,_id,Sr No.,Motor Cycles,Scooters,Moped,Cars,Jeeps,...,Private Service Vehicles,Ambulances,Articulated/Multi.,Trucks & Lorries,Tanker,Delivery Van (4 wheelers),Delivery Van (3 wheelers),Tractors,Trailors,Others
0,Amrawati Region,Akola,2000-2018,8388,8388,1831354,662478,533725,194733,67539,...,520,1862,51,65036,1730,51330,30948,116270,85159,4194
1,Amrawati Region,Amaravati,2000-2018,888,888,411115,89261,77774,47343,5528,...,139,324,8,8700,239,8990,4864,16894,9104,689
2,Amrawati Region,Amrawati,2000-2018,7446,7446,2752398,776194,1181966,284367,78668,...,883,2564,249,93044,3410,61526,46266,138294,95501,4402
3,Amrawati Region,Buldhana,2000-2018,8352,8352,2146589,207743,391400,100699,70979,...,571,2000,21,39539,931,54426,47343,208699,134752,2313
4,Amrawati Region,Washim,2000-2018,8406,8406,1026814,199646,256012,56151,40523,...,13,764,12,17109,424,23985,15356,104604,70777,557
5,Amrawati Region,Yavatmal,2000-2018,890,890,251054,51685,56859,19969,7351,...,103,131,440,5306,178,7145,3980,11296,6568,82
6,Amrawati Region,Yawatmal,2000-2018,7480,7480,1712991,456532,632721,140025,69535,...,928,1147,3750,58159,1911,44481,31814,94757,73298,819
7,Aurangabad Region,Aurangabad,2000-2018,8172,8172,6732394,1172472,821358,580450,305997,...,15024,4740,7293,159873,48293,235669,260730,277063,175665,4992
8,Aurangabad Region,Beed,2000-2018,8208,8208,1389296,174268,276431,88063,166834,...,317,161,3119,55619,4803,54490,81099,109771,79790,600
9,Aurangabad Region,Jalna,2000-2018,8190,8190,1840802,132054,168691,100090,66852,...,209,1539,939,32276,4723,37381,61173,145947,88484,3815


ii.	Write a function that would take year as parameter and plot a graph.  Classify the vehicles into 3 categories Private, Commercial and Others.  Commercial vehicles will have 2 subcategories Light Motor Vehicles (LMV) and Heavy Motor Vehicles (HMV).  Use different colors for each type of vehicle and also show the legend.  The graph should show both the values and percentages.

In [34]:
import re
import pandas as pd
import plotly.graph_objects as go
import re
import plotly.express as px

In [35]:

def get_valid_year_range():
    year_range_pattern = re.compile(r'^20(01|02|03|04|05|06|07|08|09|10|11|12|13|14|15|16|17|18)-20(02|03|04|05|06|07|08|09|10|11|12|13|14|15|16|17|18)$')

    while True:
        user_input = input("Please enter a year range in the format YYYY-YYYY (2001-2018): ")

        if year_range_pattern.match(user_input):
            start_year, end_year = map(int, user_input.split('-'))

            if start_year + 1 == end_year and 2001 <= start_year <= 2018:
                return start_year, end_year
            else:
                print("Years must be consecutive and in the range 2001-2018. Please try again.")
        else:
            print("Invalid format. Please enter a year range in the format YYYY-YYYY (2001-2018).")

def pie_graph(year):
    data = pd.read_csv('MH-Vehicle-Reg.csv', delimiter=';')

    year_data = data[data['Year'] == year]

    # Defining vehicle categories
    private_vehicles = ['Motor Cycles', 'Scooters', 'Moped', 'Cars', 'Jeeps', 'Stn. Wagons', 'Private Service Vehicles']
    commercial_vehicles = ['Taxis meter fited', 'Luxury /Turist Cabs/', 'Auto-rikshaws', 'Stage carriages', 'Contract carriages /Mini Bus', 'School Buses', 'Ambulances', 'Articulated/Multi.', 'Trucks & Lorries', 'Tanker', 'Delivery Van (4 wheelers)', 'Delivery Van (3 wheelers)', 'Tractors', 'Trailors']
    others = ['Others']
    hmvs = ['Stage carriages', 'Contract carriages /Mini Bus', 'School Buses', 'Trucks & Lorries', 'Tanker', 'Tractors', 'Trailors']
    lmvs = ['Taxis meter fited', 'Luxury /Turist Cabs/', 'Auto-rikshaws','Ambulances', 'Articulated/Multi.','Delivery Van (4 wheelers)', 'Delivery Van (3 wheelers)']

    # Calculating the total number of vehicles in each category
    private_total = year_data[private_vehicles].sum().sum()
    commercial_total = year_data[commercial_vehicles].sum().sum()
    other_total = year_data[others].sum().sum()
    lmvs_total = year_data[lmvs].sum().sum()
    hmvs_total = year_data[hmvs].sum().sum()

    private_all_category_total = year_data[private_vehicles].sum()
    private_all_category_names = list(private_all_category_total.index)
    p_len = len(private_all_category_names)

    others_all_category_total = year_data[others].sum()
    others_all_category_names = list(others_all_category_total.index)
    o_len = len(others_all_category_names)

    lmvs_all_category_total = year_data[lmvs].sum()
    lmvs_all_category_names = list(lmvs_all_category_total.index)
    l_len = len(lmvs_all_category_names)

    hmvs_all_category_total = year_data[hmvs].sum()
    hmvs_all_category_names = list(hmvs_all_category_total.index)
    h_len = len(hmvs_all_category_names)

    total = private_total + commercial_total + other_total

        #creating figure

    fig = go.Figure(go.Sunburst(
        labels=['Vehicles', 'Commercial', 'Private', 'OthersVehicles', 'LowMotor', 'HeavyMotor'] +
                private_all_category_names + others_all_category_names + lmvs_all_category_names + hmvs_all_category_names,
        parents=['', 'Vehicles', 'Vehicles', 'Vehicles', 'Commercial', 'Commercial'] +
                (['Private'] * p_len) + (['OthersVehicles'] * o_len) + (['LowMotor'] * l_len) + (['HeavyMotor'] * h_len),
        values=[total, commercial_total, private_total, other_total, lmvs_total, hmvs_total] +
                list(private_all_category_total.values) + list(others_all_category_total.values) +
                list(lmvs_all_category_total.values) + list(hmvs_all_category_total.values),
        branchvalues='total',
        marker=dict(
            colors=px.colors.sequential.Viridis
        ),
        hoverinfo='label+value+percent parent',
        textfont=dict(
            size=18,
            color='white',
            family='Open Sans',
        )
    ))

    fig.update_layout(
        title=f"Vehicle Categories Distribution for {year}",
        height=1200, width=1200,
        legend=dict(title='Vehicle Categories', traceorder='reversed'),
        plot_bgcolor='black',
    )

    fig.show()
    print(f"Plot for the year range: {year}")


start_year, end_year = get_valid_year_range()
selected_year_range = f"{start_year}-{end_year}"

pie_graph(selected_year_range)

Please enter a year range in the format YYYY-YYYY (2001-2018): 2011-2012


Plot for the year range: 2011-2012
